# import Keras, initialize conv_base

In [1]:
import keras
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
                  

C:\Users\Dennis1\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# crop Test Picture

In [6]:
from PIL import Image
from random import randint
import glob
import random
import sys
import os, shutil
# enter the path to save the tiles------------------------------------------------------------------------------------
folder = ("TEST/Tiles/")

for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
       
    except Exception as e:
        print(e)


numberTiles = 10

counter = 1
# enter the path of the image--------------------------------------------------------------------------------------------
path = "TEST/BILD.jpg"

    
    



for NumberTile in range(numberTiles):


    img = Image.open(path)
    width = img.size[0]
    height = img.size[1]


    randomSize = (randint(1000, 1600))
    randomWidth =(randint(0, width-randomSize))
    randomHeight = (randint(0, height-randomSize))




    img3 = img.crop(
        (
        randomWidth ,
        randomHeight,
        randomWidth+ randomSize,
        randomHeight+ randomSize,
    )
    )
    counter = counter + 1
    img3.save(folder+str(counter)+".jpg")
    


# run Test Picture tiles through conv_base

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
# enter the path of the directory----------------------------------------------------------------------------------------
test_dir = ("c:/Users/Dennis1/ki/TEST")
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 5
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count, 3))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
        
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels
test_features, test_labels = extract_features(test_dir, 10)

test_features = np.reshape(test_features, (10, 4 * 4 * 512))

Found 10 images belonging to 1 classes.


# load classifier

In [8]:
from keras.models import load_model

# Local.h5 File has to be saved in the project folder----------------------------------------------------------------------
model = load_model("Local.h5")

# run Data thorugh Classifier and get output

In [9]:
testPred =model.predict(test_features)

totalACC = [0,0,0]

for prime in testPred:
    counter = 0
    for Acc in prime:
        if Acc>0.30:
            totalACC[counter]= totalACC[counter]+Acc
        counter= counter+1
        
        
        
if totalACC[0]>totalACC[1] and totalACC[0]>totalACC[2]:
    print("Es handelt sich hier um Raum 1")
elif totalACC[1]>totalACC[0] and totalACC[1]>totalACC[2]:
    print("Es handelt sich hier im Raum 2")
else:
    print("Es handelt sich hier um Raum Other")

Es handelt sich hier um Raum Other


In [1]:
#print(totalACC)
#print(model.predict_classes(test_features))